#### MÓDULOS A IMPORTAR

In [1]:
# Módulos para la conversión del pdf y la creación del DataFrame
import datetime
import os
import PyPDF2 as pdf2
import csv
import pandas as pd
import numpy as np
#from functions.xmlReading import myxmlreader

# Módulos para la lectura de liquidaciones
from openpyxl import Workbook, load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

# Módulos para el procesamiento de xml's
import xmltodict as xml
from lxml import etree

import tkinter


#### DEFINICIÓN DE FUNCIONES

In [2]:
def detalle_gasto(File, RangeName):
    wb = Workbook()
    wb = load_workbook(filename=File, read_only=True, data_only=True)
    ws = wb['Caratula']
    
    cardnumber = ws[rango_a_celda(wb, 'TARJETA')].value
    
    address = list(wb.defined_names[RangeName].destinations)
    for sheetname, s_cell in address:
        s_cell = s_cell.replace('$', '')
    
    e_cell = s_cell
    s_col, s_row = coordinate_from_string(s_cell)
    e_col = s_col
    s_col = column_index_from_string(s_col)
    e_row = s_row
    
    
    cvx = wb[sheetname][e_cell].value
    
    while cvx != 'Total Liquidacion':
        #print(x)
        e_row += 1
        e_cell = str(e_col) + str(e_row)
        cvx = wb[sheetname][e_cell].value

    e_col, e_row = coordinate_from_string(e_cell)
    e_row -= 1
    e_col = 9

    detailed_range = tuple([s_row, s_col, e_row, e_col])
    
    temp_lst = []
    for row in ws.iter_rows(min_row=s_row, min_col=s_col, max_row=e_row, max_col=e_col):
        for cell in row:
            temp_lst.append(cell.value)
    
    mydet = lineas_para_csv(temp_lst, (e_col-s_col+1))
    
    for n in range(len(mydet)):
        del mydet[n][1:5]
        
    for c in range(len(mydet)):
        mydet[c].insert(0, cardnumber)
        if mydet[c][2] != None:
            mydet[c].append('CRC')
        elif mydet[c][3] != None:
            mydet[c].append('USD')
        else:
            mydet[c].append(None)
    
    return mydet

In [3]:
def rango_a_celda(WorkBook, Range_Name):
    sheet_cell = list(WorkBook.defined_names[Range_Name].destinations)
    for mysheetname, mycelladdress in sheet_cell:
        mycelladdress = mycelladdress.replace('$', '')
    return mycelladdress

In [4]:
def rango_resumen(workbook, range_name, qrows, qcols):
    address = list(workbook.defined_names[range_name].destinations)
    for sheetname, celladdress in address:
        celladdress = celladdress.replace('$', '')
    s_col, s_row = coordinate_from_string(celladdress)
    s_col = column_index_from_string(s_col)
    e_row = s_row + (qrows - 1)
    e_col = s_col + (qcols - 1)
    my_range = tuple([s_row, s_col, e_row, e_col])
    return my_range

In [5]:
def lineas_para_csv(values_list, cols): #al ejecutarla le puedo pasar qty_cols
    to_csv =[]
    lst_size = len(values_list)
    reps = lst_size//cols #Se divide por cols porque es la cantidad de columnas por registro
    start_pos = 0
    end_pos = cols
    increment = cols
    
    for i in range(reps):
        mini_lst = values_list[start_pos : end_pos]
        to_csv.append(mini_lst)
        start_pos += increment
        end_pos += increment
    return to_csv

In [6]:
def leer_archivo(path_file):
    wb = Workbook()
    wb = load_workbook(filename=path_file,
                         read_only=True,
                         data_only=True)
    # wbook.sheetnames
    ws = wb['Caratula']
    card = ws[rango_a_celda(wb, 'TARJETA')].value
    crc = ws[rango_a_celda(wb, 'TCRC')].value
    usd = ws[rango_a_celda(wb, 'TUSD')].value
    totales = (card, crc, usd)
    
    #-----------------------------------------
    #Detalle gastos
    rr = rango_resumen(wb, 'RESUMEN', qty_rows, qty_cols)
    expenses_lst = resumen_gastos(rr, ws)
    #-----------------------------------------
    
    expenses = [totales, expenses_lst]
    #del wbook
        
    return expenses
    
    #print(f'{c_file}:\n {card}\n crc: {crc}\n usd: {usd}\n')

In [7]:
def resumen_gastos(Xlrange, Wsheet):
    temp_lst = []
    for row in Wsheet.iter_rows(min_row=Xlrange[0], min_col=Xlrange[1], max_row=Xlrange[2], max_col=Xlrange[3]):
        for cell in row:
            temp_lst.append(cell.value)
    myexpenses = lineas_para_csv(temp_lst, qty_cols)
    return myexpenses

In [8]:
# Va acumulando los montos liquidados por tarjeta
def sumar_liq(from_file, id_col, id_exp):
    t_exp, d_exp = leer_archivo(from_file)
    df.at[int(t_exp[0]), df.columns[id_col]] += float(t_exp[id_exp])  #Se referencia el nombre de la columna para prevenir cambios
    df.at[int(t_exp[0]), df.columns[id_col + 1]] += float(t_exp[id_exp + 1])
    return d_exp

In [9]:
def validar_mes():
    f_cwd = os.getcwd() + '\\' + os.listdir(path=root)[1] + '\\' + month
    valid = os.path.exists(f_cwd)
    return valid

In [10]:
def extract_from_xml(my_file, doc_type):
    tree = etree.parse(my_file)
    #root = tree.getroot()
    xml_str = etree.tostring(tree)
    #data = xml.parse(xml_str, process_namespaces=True) #Esta línea procesa los namespaces...no es necesario para esto
    global data
    data = xml.parse(xml_str, dict_constructor=dict)
    for t in doc_type:
        if data.get(t) != None:
            if invoices.get(data[t]['NumeroConsecutivo']) == None:
                try:
                    Consecutivo = data[t]['NumeroConsecutivo']
                    Receptor = data[t]['Receptor']['Identificacion']['Numero']
                    Moneda = data[t]['ResumenFactura']['CodigoTipoMoneda']['CodigoMoneda']
                    Ventaneta = float(data[t]['ResumenFactura']['TotalVentaNeta'])
                    Impuesto = float(data[t]['ResumenFactura']['TotalImpuesto'])
                    Comprobante = float(data[t]['ResumenFactura']['TotalComprobante'])
                    # Aquí se construye el diccionario de cada documento
                    invoices[Consecutivo] = {'receptor' : Receptor,'moneda' : Moneda, 'ventaneta' : Ventaneta, 'impuesto' : Impuesto, 'comprobante' : Comprobante}
                    #print(f'Read: {f} at {datetime.datetime.utcnow()}')
                    read.append(f)
                    break
                except:
                    #print(f'Inner unmatched: {f} at {datetime.datetime.utcnow()}')
                    unmatched.append(f)
                    pass
            else:
                #print(f'Repeated: {f} at {datetime.datetime.utcnow()}')
                repeated.append(f)
                break
        else:
            #print(f'Outter unmatched: {f} at {datetime.datetime.utcnow()}')
            unmatched.append(f)
            break


In [11]:
def resultados_xml(fl, un, rep, re, iv):
    print('\n-- SUMARY --')
    print(f'\nFailed: {len(failed)}\nUnmatched: {len(unmatched)}\nRepeated:{len(repeated)}\nRead: {len(read)}\n')
    
    for i in fl:
        print(f'Failed reading: {i}')

    print(50 * '=' + '\n')
    for i in un:
        print(f'Unmatched xml type: {i}')

    print(50 * '=' + '\n')
    for i in rep:
        print(f'Repeated xml docx: {i}')

    print(50 * '=' + '\n')
    for i in re:
        print(f'Succesfully read: {i}')

    print(50 * '=' + '\n')
    print('Data extracted:\n')
    for i in iv.keys():
        print(f'{i} : {invoices[i]}')
        
    #print(50 * '=' + '\n')
    #print('Invoices Detail:\n')
    #for i in invoices:
    #    print(i)


In [12]:
def gasto_validado(liq_monto, xml_monto, liq_moneda, xml_moneda):
    try:
        if (liq_moneda == xml_moneda) & (abs(liq_monto - xml_monto) < 1):
            return True
        else:
            return False
    except:
        print(f'Invoice not found')
        return False

In [13]:
def closewindow():
    ventana.destroy()

#### ASIGNACIONES INICIALES

In [14]:
#pd.set_option('display.precision', 2)

In [15]:
cwd = os.getcwd()

In [16]:
cwd_file = cwd + '\\EstadoCuentaConsolidadoNaret.pdf'

In [17]:
savingfolder = '\\CreatedFiles'
savingfolder = cwd + savingfolder

In [18]:
root = cwd + '\\'
#root

##### TENER PRESENTE CUÁL SERÁ LA EXTENSIÓN DE LOS ARCHIVOS A USAR

In [19]:
file_name = '\\LiquidacionTarjeta.xlsm'

##### EJECUCIÓN

In [20]:
valid = False
while valid == False:
    month = str(input('Ingrese mes y año "Mmmm yyyy": '))
    month = month.title()
    valid = validar_mes()


Ingrese mes y año "Mmmm yyyy":  julio 2020


##### Conversión del pdf y creación del DataFrame

In [21]:
if os.path.exists(savingfolder) == False:
    os.mkdir(savingfolder)

In [22]:
#savingfolder = cwd + savingfolder  

In [23]:
f = open(cwd_file, 'rb')

In [24]:
newpdf = pdf2.PdfFileReader(f)

In [25]:
pages = newpdf.numPages

In [26]:
txt = str()

In [27]:
for i in range(pages):
    txt = txt + newpdf.getPage(i).extractText()

In [28]:
cleaned = txt

In [29]:
clean_a = cleaned.find('Total en consumos')

In [30]:
cleaned = cleaned[ : clean_a]

In [31]:
clean_bx = cleaned.find('\nConsolidado - ')

In [32]:
clean_by = cleaned.find('\n', clean_bx+1)
clean_by = cleaned.find('\n', clean_by+1)
clean_by += 1

In [33]:
clean_b = cleaned[clean_bx : clean_by]

In [34]:
toclean = ['\nCuenta: 4116110005410\n', clean_b]
for i in toclean:
    cleaned = cleaned.replace(i, '')

In [35]:
cleaned = cleaned.replace('\nColones\n\n', '\nColones\nDolares\n')
cleaned = cleaned.replace('\n', ';')
cleaned = cleaned.replace(',', '')

In [36]:
lst_cleaned = cleaned.split(';')

In [37]:
final_lst = lineas_para_csv(lst_cleaned, 4)

In [38]:
# PARA VER LA LISTA FINAL CON QUE SE CONSTRUIRÁ EL CSV
#final_lst

In [39]:
with open(savingfolder + '\\CorteTarjetas.csv', 'w') as file:
    writer = csv.writer(file)
    for i in final_lst:
        writer.writerow(i)


In [40]:
df = pd.read_csv(savingfolder + '\\CorteTarjetas.csv')

In [41]:
os.remove(savingfolder + '\\CorteTarjetas.csv')

In [42]:
df['Colones'] = df['Colones'].astype('float')

df['Dolares'] = df['Dolares'].astype('float')

df['Tarjeta'] = df['Tarjeta'].astype('object')

In [43]:
df.rename(columns = {'Colones' : 'Corte_Colones', 'Dolares' : 'Corte_Dolares'}, inplace=True)

In [44]:
df.set_index('Tarjeta', inplace=True)

In [45]:
df['Liq_Colones'] = 0
df['Liq_Colones'] = df['Liq_Colones'].astype('float')

In [46]:
df['Liq_Dolares'] = 0
df['Liq_Dolares'] = df['Liq_Dolares'].astype('float')

In [47]:
cc = df['Corte_Colones'].sum()
cd = df['Corte_Dolares'].sum()

In [48]:
print('- - - CREDIT CARD STATEMENT HAS BEEN COMPLETED - - -')

- - - CREDIT CARD STATEMENT HAS BEEN COMPLETED - - -


## AQUI INICA LA LECTURA DE LAS LIQUIDACIONES

In [49]:
print("- - - STARTING EXPENSES' REPORTS READING - - -")

- - - STARTING EXPENSES' REPORTS READING - - -


In [50]:
#print('Dimensiones del rango del resumen\n')
qty_rows = 10
qty_cols = 3
#qty_rows = int(input('Filas: '))
#qty_cols = int(input('Columnas: '))
#print(f'{qty_rows} filas x {qty_cols} columnas')

In [51]:
path2 = '\\' + month + '\\Gastos tarjeta de credito\\'
#path2

In [52]:
# Lista de condominios
condos = os.listdir(path=root)
#condos

## Tener presente que en producción esta carpeta no debería tener nada más que carpetas de condominios

In [53]:
groups_path = root + condos[1] + path2
#groups_path

In [54]:
groups = os.listdir(path=groups_path)
#groups

In [55]:
people_path = root + 'NARET'
#people_path

In [56]:
people = os.listdir(path=people_path)
#people

In [57]:
det_lst = []

In [58]:
invoices = dict()
failed = list()
unmatched = list()
read = list()
repeated = list()
doctype = tuple(['FacturaElectronica', 'TiqueteElectronico'])
inv_liq = []
prev_inv_liq = []

In [59]:
for c in condos:
    if c != 'NARET':
        midpath = root + c + path2
        for g in groups:
            fullpath = midpath + g
            c_file = fullpath + file_name
            try:
                v = sumar_liq(c_file, 3, 1)
                det_lst.append(v)
                #print(f'a) {v}')
                #inv_liq.append(detalle_gasto(c_file, 'DETALLE'))
            except:
                #print(f'a) Error reading {c_file}\n')
                pass

            if g == groups[0]:
                asst_path = fullpath + '\\Asistente'
                #print(asst_path)
                if os.path.exists(asst_path):
                    c_file = asst_path + file_name
                    try:
                        v = sumar_liq(c_file, 3, 1)
                        det_lst.append(v)
                        #print(f'Lectura y actualización de archivo de Asistente de {c} correcta \n')
                        #inv_liq.append(detalle_gasto(c_file, 'DETALLE'))
                    except:
                        pass
                        #print(f'No se puedo leer archivo de Asistente {c}\n')
                #else:
                #    print(f'Invalid path: {asst_path}\n')

    else:
        midpath = root + 'NARET\\'
        for p in people:
            fullpath = midpath + p + '\\' + month + '\\Gastos tarjeta de credito'
            if os.path.isdir(fullpath):
                inv_lst = os.listdir(fullpath)
                #print(f'Actual inv_lst {inv_lst}')
                for f in inv_lst:
                    try:
                        myxmlfile = fullpath + '\\' + f
                        extract_from_xml(myxmlfile, doctype)
                    except:
                        #print(f'Failed: {f} at {datetime.datetime.utcnow()}')
                        failed.append(f)
                c_file = fullpath + file_name
                try:
                    v = sumar_liq(c_file, 3, 1)
                    det_lst.append(v)
                    #print(f'b) {v}')
                    print(f'Se agregará un detalle por facturas a la lista')
                    prev_inv_liq.append(detalle_gasto(c_file, 'DETALLE'))
                except:
                    #print(f'b) Error reading {c_file}\n')
                    pass
            
        
        # PUNTO DE CONTROL: Imprime las rutas y los archivos en cada ruta
        # print('\n', fullpath)
        # print('\n', fullpath, '\n', '\t', os.listdir(fullpath),'\n')

In [60]:
df['Dif_Colones'] = df['Liq_Colones'] - df['Corte_Colones']
df['Dif_Dolares'] = df['Liq_Dolares'] - df['Corte_Dolares']

##### Finalmente, guardar toda la información en Excel y eliminar archivos temporales

In [61]:
df.to_csv(savingfolder + '\\ConsumosTarjetas.csv')

In [62]:
# Una opción de to_csv si la configuración de la máquina tiene coma como decimal
df.to_csv(savingfolder + '\\ConsumosTarjetas_comas.csv', sep=';', decimal=',')

In [63]:
print(df)

                           Beneficiario  Corte_Colones  Corte_Dolares  \
Tarjeta                                                                 
4116110005410089   ANTHONY MADRIZ BEITA      678262.22           0.00   
4116110005410147      ISIDRO ALARCON S.      426399.58           0.00   
4116110005410154      JOHAN ARGUEDAS R.       18405.00           0.00   
4116110005410188   JOSE AGUERO GONZALEZ      799970.97           0.00   
4116110005410204         JULIAN VEGA V.       44195.00           0.00   
4116110005410212    KENNY HAGES EDWARDS         420.00           0.00   
4116110005410220  LUIS AGUILAR BOGANTES       56063.00           0.00   
4116110005410238    LUIS SEGURA RAMIREZ      498993.26           0.00   
4116110005410295       JOSE MURRAY DIAZ      635810.79           1.13   
4116110005410311      FABRICIO GOMEZ C.           0.00          11.30   
4116110005410329      DIEGO ROJAS CALVO     1064854.69           0.00   
4116110005410378      ANA MOLINA BRENES         840

In [64]:
lc = df['Liq_Colones'].sum()
ld = df['Liq_Dolares'].sum()

In [65]:
#Aquí estaba la impresión de los totales de liquidaciones

In [66]:
#det_lst

#### CONSTRUIR CSV Y DATAFRAME CON EL DETALLE DE TODAS LAS CUENTAS DE GASTO

In [67]:
print('INICIANDO CONSTRUCCION DE DATA FRAME GENERAL COMPARATIVO POR TARJETAS')

INICIANDO CONSTRUCCION DE DATA FRAME GENERAL COMPARATIVO POR TARJETAS


In [68]:
with open(savingfolder + '\\DetalleGastos.csv', 'w') as exp_file:
    writer = csv.writer(exp_file)
    for liq in det_lst:
        for line in liq:
            writer.writerow(line)


In [69]:
df_det = pd.read_csv(savingfolder + '\\DetalleGastos.csv')

In [70]:
os.remove(savingfolder + '\\DetalleGastos.csv')

In [71]:
print("- - - EXPENSES' REPORTS READING HAS BEEN COMPLETED - - -")

- - - EXPENSES' REPORTS READING HAS BEEN COMPLETED - - -


In [72]:
df_det.columns = ['Cuenta', 'Gasto_Colones', 'Gasto_Dolares']

In [73]:
df_det['valid'] = df_det['Gasto_Colones'] + df_det['Gasto_Dolares']

In [74]:
#df_det

In [75]:
filt = df_det['valid'] > 0

In [76]:
df_det = df_det.loc[filt]

In [77]:
df_det.reset_index(drop=True, inplace=True)

In [78]:
del df_det['valid']

In [79]:
dc = df_det['Gasto_Colones'].sum()
dd = df_det['Gasto_Dolares'].sum()

In [80]:
print(f'Totales según estado de cuenta\n\tColones: {cc}\n\tDolares: {cd}')

Totales según estado de cuenta
	Colones: 16930046.71
	Dolares: 557.49


In [81]:
print(f'Totales generales de liquidaciones procesadas\n\tColones: {dc}\n\tDolares: {dd}')

Totales generales de liquidaciones procesadas
	Colones: 3308227.45
	Dolares: 968.905


In [82]:
# La intención de esto era mostrar el total de las facturas 1 a 1
#print(f'Totales detallados de liquidaciones procesadas\n\tColones: {dc}\n\tDolares: {dd}')

In [83]:
df_det.to_excel(savingfolder + '\\GastosDetalladosPorCuenta.xlsx')

In [84]:
print("- - - CREDIT CARD AND EXPENSES' REPORTS SUCCESSFULLY PROCESSED - - -")

- - - CREDIT CARD AND EXPENSES' REPORTS SUCCESSFULLY PROCESSED - - -


#### ACÁ SE CORRE EL CÓDIGO DE LECTURA DE XML's

In [85]:
print(resultados_xml(failed, unmatched, repeated, read, invoices))


-- SUMARY --

Failed: 12
Unmatched: 0
Repeated:2
Read: 10

Failed reading: 2DEC8100
Failed reading: 8B602000
Failed reading: B8CC5000
Failed reading: DF443000
Failed reading: LiquidacionTarjeta.xlsm
Failed reading: LiquidacionTarjeta.xlsm
Failed reading: 0A23C000
Failed reading: 6AF3B000
Failed reading: 79C7D000
Failed reading: 8DA91000
Failed reading: LiquidacionTarjeta.xlsm
Failed reading: LiquidacionTarjeta_b.xlsm


Repeated xml docx: Factura_XML_3101143188_3101703823_00100001010000205641.xml
Repeated xml docx: 50604072000011013026500100006010000000055101002301_SF.xml

Succesfully read: 10-F-1030161837-1029627251.xml
Succesfully read: 50609072000011013026500100006010000000069101002128_SF - MODIF.xml
Succesfully read: Factura_XML_3101143188_3101703823_00100001010000205641 - REPETIDO.xml
Succesfully read: FE_50601062000310161019800100024010010275938200000000.xml
Succesfully read: T1_C50604082000310100975800203001010000083858100099537_E3101009758_R3101703823_F20200804.xml
Succesfully 

In [86]:
print("- - - ALL XMLs HAS BEEN READ AND LOADED INTO CURRENT PROCESS - - -\n\n")

- - - ALL XMLs HAS BEEN READ AND LOADED INTO CURRENT PROCESS - - -




In [87]:
print("- - - STARTING NARET'S EXPENSES VALIDATION AGAINST PROCESSED XMLs - - -\n\n")

- - - STARTING NARET'S EXPENSES VALIDATION AGAINST PROCESSED XMLs - - -




##### ACA INICIA EL PROCESAMIENTO DE LOS DETALLES DE GASTOS DE CADA LIQUIDACION (respaldos en cada capeta de usuario) 

#### Esto es una lista de listas, con elementos por cada liquidacion que leyó; lo que se hace en el paso siguiente es tomar esa lista de listas y extraer todos los elementos para ponerlos todos en una única lista con la info de cada línea como elemento

In [88]:
print(prev_inv_liq)

[[['4116110005410089', '00100006010000000079', 1528148.5, None, 'CRC'], ['4116110005410089', '00100001010000205641', 39316.33, None, 'CRC'], ['4116110005410089', '00100024010010275938', 168722.88, None, 'CRC'], ['4116110005410089', '00203001010000083858', 19774, None, 'CRC'], ['4116110005410089', 44018, 0.1, None, 'CRC'], ['4116110005410089', 'BLANK', 0.2, None, 'CRC'], ['4116110005410089', '00300003010000094035', None, 273, 'USD'], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None]], [['4116110005410154', '00100006010000000055', 927, None, 'CRC'], ['4116110005410154', '00100006010000000023', 14600, None, 'CRC'], ['411611000541015

In [89]:
for l in range(len(prev_inv_liq)):
    slices = len(prev_inv_liq[l])
    for s in range(slices):
        inv_liq.append(prev_inv_liq[l][s])
#inv_liq

In [90]:
print(inv_liq)

[['4116110005410089', '00100006010000000079', 1528148.5, None, 'CRC'], ['4116110005410089', '00100001010000205641', 39316.33, None, 'CRC'], ['4116110005410089', '00100024010010275938', 168722.88, None, 'CRC'], ['4116110005410089', '00203001010000083858', 19774, None, 'CRC'], ['4116110005410089', 44018, 0.1, None, 'CRC'], ['4116110005410089', 'BLANK', 0.2, None, 'CRC'], ['4116110005410089', '00300003010000094035', None, 273, 'USD'], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410089', None, None, None, None], ['4116110005410154', '00100006010000000055', 927, None, 'CRC'], ['4116110005410154', '00100006010000000023', 14600, None, 'CRC'], ['4116110005410154',

In [91]:
#inv_liq = detalle_gasto('C:\\Users\\HPC\\Documents\\Condominios\\NARET\\Paco\\Julio 2020\\Gastos tarjeta de credito\\LiquidacionTarjeta.xlsm', 'DETALLE')

In [92]:
inv = pd.DataFrame(inv_liq, columns=['cardNumber', 'invID', 'liqCRC', 'liqUSD', 'liqMoneda'])
#inv

In [93]:
filt = inv['liqMoneda'].isnull()
#filt

In [94]:
df_inv = pd.DataFrame(inv.loc[~filt])

In [95]:
inv

,cardNumber,invID,liqCRC,liqUSD,liqMoneda
0,4116110005410089,00100006010000000079,1528148.50,NaN,CRC
1,4116110005410089,00100001010000205641,39316.33,NaN,CRC
2,4116110005410089,00100024010010275938,168722.88,NaN,CRC
3,4116110005410089,00203001010000083858,19774.00,NaN,CRC
4,4116110005410089,44018,0.10,NaN,CRC
5,4116110005410089,BLANK,0.20,NaN,CRC
6,4116110005410089,00300003010000094035,NaN,273.000,USD
7,4116110005410089,None,NaN,NaN,None
8,4116110005410089,None,NaN,NaN,None
9,4116110005410089,None,NaN,NaN,None


In [96]:
df_inv['liqCRC'].replace(to_replace=np.nan, value=0, inplace=True)

In [97]:
df_inv['liqUSD'].replace(to_replace=np.nan, value=0, inplace=True)

In [98]:
df_inv['liqMonto'] = df_inv['liqCRC'] + df_inv['liqUSD']

In [99]:
df_inv['invID'] = df_inv['invID'].astype('str')

In [100]:
df_inv['validinvID'] = df_inv.apply(lambda x: len(x['invID']), axis=1)

In [101]:
filt_invID = df_inv['validinvID'] == 20
#filt_invID

In [102]:
df_inv_invalid = df_inv.loc[~filt_invID]

In [103]:
df_inv_invalid.to_csv(savingfolder + '\\GastosErrorNumeroFactura.csv')

In [104]:
df_inv = df_inv.loc[filt_invID]

In [105]:
del df_inv['validinvID']

In [106]:
df_inv.set_index('invID', inplace=True)

In [107]:
df_inv['xmlMoneda'] = None
df_inv['xmlMonto'] = None

In [108]:
invids = df_inv.index.tolist()

In [109]:
invids

['00100006010000000079',
 '00100001010000205641',
 '00100024010010275938',
 '00203001010000083858',
 '00300003010000094035',
 '00100006010000000055',
 '00100006010000000023',
 '00100001010000001500',
 '00100001010000001498',
 '00100001010000001497']

In [110]:
#df_inv

In [111]:
for i in invids:
    try:
        df_inv.at[i, 'xmlMonto'] = invoices[i]['comprobante']
        df_inv.at[i, 'xmlMoneda'] = invoices[i]['moneda']
    except:
        print(f'Invoice number {i} not found')
        continue

In [112]:
#del df_inv['liqCRC']
#del df_inv['liqUSD']

In [113]:
#df_inv['validated'] = None

In [114]:
df_inv

,cardNumber,liqCRC,liqUSD,liqMoneda,liqMonto,xmlMoneda,xmlMonto
invID,,,,,,,
00100006010000000079,4116110005410089,1528148.50,0.000,CRC,1528148.500,CRC,1.52815e+06
00100001010000205641,4116110005410089,39316.33,0.000,CRC,39316.330,CRC,39316.3
00100024010010275938,4116110005410089,168722.88,0.000,CRC,168722.880,CRC,168723
00203001010000083858,4116110005410089,19774.00,0.000,CRC,19774.000,CRC,19775
00300003010000094035,4116110005410089,0.00,273.000,USD,273.000,USD,273
00100006010000000055,4116110005410154,927.00,0.000,CRC,927.000,CRC,927
00100006010000000023,4116110005410154,14600.00,0.000,CRC,14600.000,CRC,14600
00100001010000001500,4116110005410154,0.00,59.325,USD,59.325,USD,59.325
00100001010000001498,4116110005410154,0.00,98.875,USD,98.875,USD,98.875


In [115]:
df_inv['validada'] = df_inv.apply(lambda x: gasto_validado(x['liqMonto'], x['xmlMonto'], x['liqMoneda'], x['xmlMoneda']), axis=1)
df_inv

,cardNumber,liqCRC,liqUSD,liqMoneda,liqMonto,xmlMoneda,xmlMonto,validada
invID,,,,,,,,
00100006010000000079,4116110005410089,1528148.50,0.000,CRC,1528148.500,CRC,1.52815e+06,False
00100001010000205641,4116110005410089,39316.33,0.000,CRC,39316.330,CRC,39316.3,True
00100024010010275938,4116110005410089,168722.88,0.000,CRC,168722.880,CRC,168723,True
00203001010000083858,4116110005410089,19774.00,0.000,CRC,19774.000,CRC,19775,False
00300003010000094035,4116110005410089,0.00,273.000,USD,273.000,USD,273,True
00100006010000000055,4116110005410154,927.00,0.000,CRC,927.000,CRC,927,True
00100006010000000023,4116110005410154,14600.00,0.000,CRC,14600.000,CRC,14600,True
00100001010000001500,4116110005410154,0.00,59.325,USD,59.325,USD,59.325,True
00100001010000001498,4116110005410154,0.00,98.875,USD,98.875,USD,98.875,True


In [116]:
validated = df_inv.index.tolist()

In [117]:
for i in validated:
    try:
        invoices[i]['validada'] = True
    except:
        print(f"There's no an XML for invoice {i}")
        continue

In [118]:
#final_df = df_inv.groupby('cardNumber')

In [119]:
df_inv.to_csv(savingfolder + '\\GastosPorFacturaValidados.csv')

In [120]:
#invoices

In [121]:
ventana = tkinter.Tk()
ventana.geometry('300x100')
ventana.title('Proccess Completed')

mensaje = tkinter.Label(text='Accomplished!! See results at "CreatedFiles" folder')
mensaje.grid(row=0, column=0, columnspan=True)

aceptar = tkinter.Button(ventana, text='Finish', command=closewindow, padx=15, pady=5)
aceptar.pack(side='bottom')

ventana.mainloop()